# Preprocess Dataset

## Setup

In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import json
import re
from email.parser import Parser
from email import policy
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## Download Dataset

In [2]:
# Directory structure
RAW_DATA_DIR = Path("data/raw")
DATA_PREPROCESSED_DIR = Path("data/preprocessed")

# Create directories
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
DATA_PREPROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Dataset names on Kaggle
PHISHING_DATASET = "subhajournal/phishingemails"
ENRON_DATASET = "wcukierski/enron-email-dataset"

def download_datasets():
    """Download datasets from Kaggle if not already present."""
    
    # Download phishing dataset
    phishing_dir = RAW_DATA_DIR / "phishing"
    phishing_dir.mkdir(exist_ok=True)
    
    if not any(phishing_dir.iterdir()):
        print(f"Downloading phishing dataset: {PHISHING_DATASET}...")
        os.system(f"kaggle datasets download -d {PHISHING_DATASET} -p {phishing_dir} --unzip")
        print("✅ Phishing dataset downloaded!")
    else:
        print("✅ Phishing dataset already exists. Skipping download.")
    
    # Download Enron dataset
    enron_dir = RAW_DATA_DIR / "enron"
    enron_dir.mkdir(exist_ok=True)
    
    if not any(enron_dir.iterdir()):
        print(f"Downloading Enron dataset: {ENRON_DATASET}...")
        os.system(f"kaggle datasets download -d {ENRON_DATASET} -p {enron_dir} --unzip")
        print("✅ Enron dataset downloaded!")
    else:
        print("✅ Enron dataset already exists. Skipping download.")


download_datasets()

✅ Phishing dataset already exists. Skipping download.
✅ Enron dataset already exists. Skipping download.


## Setup

In [3]:
def load_phishing_dataset():
    """Load the phishing email dataset."""
    
    phishing_dir = RAW_DATA_DIR / "phishing"
    
    # Load the CSV file
    df = pd.read_csv(phishing_dir / "Phishing_Email.csv")

    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"First few rows:")
    print(df.head(3))
    
    # Check for label distribution
    print(f"Label distribution:")
    print(df['Email Type'].value_counts())
    
    return df

def load_enron_dataset(sample_size=5000):
    """Load a sample of the Enron email dataset.
    
    Args:
        sample_size: Number of emails to sample (Enron is very large)
    """
    enron_dir = RAW_DATA_DIR / "enron"
    
    df = pd.read_csv(enron_dir / "emails.csv")
  
    print(f"Full dataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    
    # Sample the dataset
    if len(df) > sample_size:
        print(f"Sampling {sample_size} emails from {len(df)} total emails...")
        df = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

    print(f"Sampled dataset shape: {df.shape}")
    print(f"First few rows:")
    print(df.head(3))
    
    return df

print("PHISHING DATASET:")
phishing_df = load_phishing_dataset()

print("\nENRON DATASET:")
enron_df = load_enron_dataset(sample_size=5000)

PHISHING DATASET:
Dataset shape: (18650, 3)
Columns: ['Unnamed: 0', 'Email Text', 'Email Type']
First few rows:
   Unnamed: 0                                         Email Text  Email Type
0           0  re : 6 . 1100 , disc : uniformitarianism , re ...  Safe Email
1           1  the other side of * galicismos * * galicismo *...  Safe Email
2           2  re : equistar deal tickets are you still avail...  Safe Email
Label distribution:
Email Type
Safe Email        11322
Phishing Email     7328
Name: count, dtype: int64

ENRON DATASET:
Full dataset shape: (517401, 2)
Columns: ['file', 'message']
Sampling 5000 emails from 517401 total emails...
Sampled dataset shape: (5000, 2)
First few rows:
                         file  \
0     shackleton-s/sent/1912.   
1    farmer-d/logistics/1066.   
2  parks-j/deleted_items/202.   

                                             message  
0  Message-ID: <21013688.1075844564560.JavaMail.e...  
1  Message-ID: <22688499.1075854130303.JavaMail.e...  
2 

## Preprocess Datasets

In [4]:
def clean_text(text):
    """Clean email text."""
    if pd.isna(text):
        return ""
    
    text = str(text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def parse_email_body_to_str(text):
    """Extract email body from raw email text if needed."""
    if pd.isna(text):
        return ""

    # Try to parse as email if it looks like raw email format
    if "Subject:" in text or "From:" in text:
        try:
            parser = Parser(policy=policy.default)
            msg = parser.parsestr(text)
            
            # Get email body
            if msg.is_multipart():
                body = ""
                for part in msg.walk():
                    if part.get_content_type() == "text/plain":
                        body += part.get_payload(decode=True).decode('utf-8', errors='ignore')
            else:
                body = msg.get_payload(decode=True).decode('utf-8', errors='ignore')
            
            return body
        except:
            return text
        
    return text
    
def remove_short_emails(df, min_length=5):
    """Remove emails with text shorter than min_length."""
    initial_count = len(df)
    df = df[df['text'].str.len() >= min_length].reset_index(drop=True)
    final_count = len(df)
    print(f"Removed {initial_count - final_count} short emails (length < {min_length})")
    return df

def preprocess_phishing_dataset(df):
    """Preprocess the phishing dataset to standard format."""
    
    # Create a standardized dataframe
    processed_df = pd.DataFrame()
    
    # Try to identify text and label columns
    processed_df['text'] = df['Email Text']
    
    # Extract label
    # Map Email Type to binary label (0: legitimate, 1: phishing)
    processed_df['label'] = df['Email Type'].map({
        'Safe Email': 0,
        'Phishing Email': 1
    })
   
    
    # Clean text
    processed_df['text'] = processed_df['text'].apply(parse_email_body_to_str)
    processed_df['text'] = processed_df['text'].apply(clean_text)
    processed_df = remove_short_emails(processed_df, min_length=2)

    
    print(f"Processed {len(processed_df)} emails")
    print(f"Label distribution:")
    print(processed_df['label'].value_counts())
    
    return processed_df

def preprocess_enron_dataset(df):
    """Preprocess the Enron dataset to standard format."""
    
    # Create a standardized dataframe
    processed_df = pd.DataFrame()
    
    # Try to identify text column
    processed_df['text'] = df['message']
   
    
    # All Enron emails are legitimate (label = 0)
    processed_df['label'] = 0
    
    # Clean text
    print("Cleaning text...")
    processed_df['text'] = processed_df['text'].apply(parse_email_body_to_str)
    processed_df['text'] = processed_df['text'].apply(clean_text)

    processed_df = remove_short_emails(processed_df, min_length=2)


    print(f"Processed {len(processed_df)} legitimate emails")

    return processed_df

print("Phishing Dataset Preprocessing:")
phishing_processed_df = preprocess_phishing_dataset(phishing_df)

print("\nEnron Dataset Preprocessing:")
enron_processed_df = preprocess_enron_dataset(enron_df)


Phishing Dataset Preprocessing:
Removed 38 short emails (length < 2)
Processed 18612 emails
Label distribution:
label
0    11309
1     7303
Name: count, dtype: int64

Enron Dataset Preprocessing:
Cleaning text...
Removed 3 short emails (length < 2)
Processed 4997 legitimate emails


## Feature Engineering

In [5]:
import nltk
from nltk.corpus import stopwords

try:
        STOPWORDS = set(stopwords.words('english'))
except:
    print("⚠️ Downloading NLTK stopwords...")
    nltk.download('stopwords', quiet=True)
    STOPWORDS = set(stopwords.words('english'))

from spellchecker import SpellChecker

spell = SpellChecker()

# Urgent keywords for phishing detection
URGENT_KEYWORDS = [
    'urgent', 'immediately', 'action required', 'verify', 'confirm', 
    'suspend', 'restricted', 'expired', 'update', 'click here',
    'act now', 'limited time', 'alert', 'warning', 'attention',
    'security', 'unusual activity', 'locked', 'expiring', 'renewal'
]

def extract_num_words(text):
    """Extract total number of words."""
    if pd.isna(text) or text == "":
        return 0
    words = text.split()
    return len(words)

def extract_num_unique_words(text):
    """Extract count of unique words."""
    if pd.isna(text) or text == "":
        return 0
    words = [word.lower() for word in text.split()]
    return len(set(words))

def extract_num_stopwords(text):
    """Extract count of stopwords."""
    if pd.isna(text) or text == "":
        return 0
    words = [word.lower() for word in text.split()]
    return sum(1 for word in words if word in STOPWORDS)

def extract_num_links(text):
    """Extract number of hyperlinks."""
    if pd.isna(text) or text == "":
        return 0
    # Match URLs (http, https, www)
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    links = re.findall(url_pattern, text)
    return len(links)

def extract_num_unique_domains(text):
    """Extract number of unique domains in links."""
    if pd.isna(text) or text == "":
        return 0
    # Match URLs and extract domains
    url_pattern = r'http[s]?://([a-zA-Z0-9.-]+)|www\.([a-zA-Z0-9.-]+)'
    matches = re.findall(url_pattern, text)
    domains = [match[0] if match[0] else match[1] for match in matches]
    domains = [d.split('/')[0] for d in domains if d]  # Clean up domains
    return len(set(domains))

def extract_num_email_addresses(text):
    """Extract count of email addresses."""
    if pd.isna(text) or text == "":
        return 0
    # Match email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(email_pattern, text)
    return len(emails)

def extract_num_spelling_errors(text):
    """Extract count of misspelled words."""
    if pd.isna(text) or text == "":
        return 0
    
    # Extract words (alphanumeric only)
    words = re.findall(r'\b[a-zA-Z]+\b', text.lower())
    
    # Filter out very short words and check spelling
    words = [w for w in words if len(w) > 2]
    
    # Limit to first 100 words for performance
    words = words[:100]
    
    misspelled = spell.unknown(words)
    return len(misspelled)

def extract_num_urgent_keywords(text):
    """Extract count of urgent keywords."""
    if pd.isna(text) or text == "":
        return 0
    
    text_lower = text.lower()
    count = 0
    for keyword in URGENT_KEYWORDS:
        count += text_lower.count(keyword)
    
    return count

def extract_features(df):
    """Extract all features from the text column."""
    
    print("="*70)
    print("FEATURE ENGINEERING")
    print("="*70)
    
    print("\n🔧 Extracting features...")
    
    # Extract each feature
    print("   ✓ Extracting num_words...")
    df['num_words'] = df['text'].apply(extract_num_words)
    
    print("   ✓ Extracting num_unique_words...")
    df['num_unique_words'] = df['text'].apply(extract_num_unique_words)
    
    print("   ✓ Extracting num_stopwords...")
    df['num_stopwords'] = df['text'].apply(extract_num_stopwords)
    
    print("   ✓ Extracting num_links...")
    df['num_links'] = df['text'].apply(extract_num_links)
    
    print("   ✓ Extracting num_unique_domains...")
    df['num_unique_domains'] = df['text'].apply(extract_num_unique_domains)
    
    print("   ✓ Extracting num_email_addresses...")
    df['num_email_addresses'] = df['text'].apply(extract_num_email_addresses)
    
    print("   ✓ Extracting num_spelling_errors (this may take a while)...")
    df['num_spelling_errors'] = df['text'].apply(extract_num_spelling_errors)
    
    print("   ✓ Extracting num_urgent_keywords...")
    df['num_urgent_keywords'] = df['text'].apply(extract_num_urgent_keywords)
    
    print("\n✅ Feature extraction complete!")
    print("\n" + "="*70 + "\n")
    
    return df


## Combine Datasets and Save

In [6]:
def combine_and_save(phishing_df, enron_df):
    """Combine datasets and save to CSV."""

    # Combine datasets
    combined_df = pd.concat([phishing_df, enron_df], ignore_index=True)
    
    # Shuffle the dataset
    combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f"Combined dataset shape: {combined_df.shape}")
    print(f"Final label distribution:")
    print(combined_df['label'].value_counts())
    print(f"Class balance:")
    print(combined_df['label'].value_counts(normalize=True))

    # Extract features
    combined_df = extract_features(combined_df)

    # Reorder columns: text, label, then all features
    feature_cols = [
        'num_words', 'num_unique_words', 'num_stopwords', 'num_links',
        'num_unique_domains', 'num_email_addresses', 'num_spelling_errors',
        'num_urgent_keywords'
    ]
    
    combined_df = combined_df[['text', 'label'] + feature_cols]
    
    # Save to CSV
    output_file = DATA_PREPROCESSED_DIR / "emails_combined.csv"
    combined_df.to_csv(output_file, index=False)
    print(f"Saved combined dataset to: {output_file}")
    print(f"Final shape: {combined_df.shape}")
    print(f"Columns: {combined_df.columns.tolist()}")

    # Show sample emails
    print("\n\nSAMPLE DATA")
    print("Sample legitimate email with features:")
    legit_sample = combined_df[combined_df['label'] == 0].iloc[0]
    print(f"Text (first 300 chars): {legit_sample['text'][:300]}...")
    print(f"\nFeatures:")
    for col in feature_cols:
        print(f"  {col}: {legit_sample[col]}")

    print(f"\nSample phishing email with features:")
    phish_sample = combined_df[combined_df['label'] == 1].iloc[0]
    print(f"Text (first 300 chars): {phish_sample['text'][:300]}...")
    print(f"\nFeatures:")
    for col in feature_cols:
        print(f"  {col}: {phish_sample[col]}")
    
    
    return combined_df


# Combine and save
combined_df = combine_and_save(phishing_processed_df, enron_processed_df)

Combined dataset shape: (23609, 2)
Final label distribution:
label
0    16306
1     7303
Name: count, dtype: int64
Class balance:
label
0    0.690669
1    0.309331
Name: proportion, dtype: float64
FEATURE ENGINEERING

🔧 Extracting features...
   ✓ Extracting num_words...
   ✓ Extracting num_unique_words...
   ✓ Extracting num_stopwords...
   ✓ Extracting num_links...
   ✓ Extracting num_unique_domains...
   ✓ Extracting num_email_addresses...
   ✓ Extracting num_spelling_errors (this may take a while)...
   ✓ Extracting num_urgent_keywords...

✅ Feature extraction complete!


Saved combined dataset to: data/preprocessed/emails_combined.csv
Final shape: (23609, 10)
Columns: ['text', 'label', 'num_words', 'num_unique_words', 'num_stopwords', 'num_links', 'num_unique_domains', 'num_email_addresses', 'num_spelling_errors', 'num_urgent_keywords']


SAMPLE DATA
Sample legitimate email with features:
Text (first 300 chars): Are these students first or second years? If they are first years, we

In [7]:
feature_cols = [
        'num_words', 'num_unique_words', 'num_stopwords', 'num_links',
        'num_unique_domains', 'num_email_addresses', 'num_spelling_errors',
        'num_urgent_keywords'
    ]

print("\n📊 Feature statistics by class:")
print("\n" + "="*70)
print("LEGITIMATE EMAILS (label=0)")
print("="*70)
print(combined_df[combined_df['label'] == 0][feature_cols].describe())

print("\n" + "="*70)
print("PHISHING EMAILS (label=1)")
print("="*70)
print(combined_df[combined_df['label'] == 1][feature_cols].describe())


📊 Feature statistics by class:

LEGITIMATE EMAILS (label=0)
          num_words  num_unique_words  num_stopwords     num_links  \
count  16306.000000      16306.000000   16306.000000  16306.000000   
mean     342.292714        151.941555     101.779406      0.883908   
std      870.759412        227.142709     278.699913     24.726966   
min        1.000000          1.000000       0.000000      0.000000   
25%       65.000000         50.000000      18.000000      0.000000   
50%      155.000000        100.000000      47.000000      0.000000   
75%      345.000000        178.000000     103.000000      1.000000   
max    23343.000000       7700.000000    7338.000000   3133.000000   

       num_unique_domains  num_email_addresses  num_spelling_errors  \
count        16306.000000         16306.000000         16306.000000   
mean             0.627008             0.849565             5.772415   
std             18.843671             6.219692             5.685461   
min              0.00000